In [1]:
# this is a copy of tutorial 3 at the start
# but instead of faking it by drogging the car through waypoints 
# we will be giving the car simple inputs to guide it along the route

In [2]:
# all tsports 
import carla # the sim library itself 
import time # to set a delay after each photo 
import cv2 # to work with images from cameras
import numpy as np # in this example to change image representation re-shaping 
import math

In [3]:
#connect to the sim

client = carla.Client('localhost', 2000)

In [4]:
# Define the environment/world and get possible spawn points
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)  # Set timeout to connect to the server
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

# Look for a blueprint of a Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

# Select a spawn point and spawn the vehicle
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

# Check if the vehicle was spawned successfully
if vehicle is None:
    print("Failed to spawn vehicle. Check spawn point or blueprint.")
else:
    print("Vehicle spawned successfully!")

# Camera mount offset on the car
CAMERA_POS_Z = 3.0
CAMERA_POS_X = -5.0

# Setting up the RGB camera
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))

# Create the camera in the simulator and attach it to the vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# Camera callback function to process images
def camera_callback(image, data_dict):
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    reshaped = array.reshape((image.height, image.width, 4))  # BGRA format
    data_dict['image'] = reshaped

# Camera resolution attributes
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

# Initialize the camera data dictionary
camera_data = {'image': np.zeros((image_h, image_w, 4))}

# Start the camera and listen for images
camera.listen(lambda image: camera_callback(image, camera_data))
print("Camera is streaming live images.")


Vehicle spawned successfully!
Camera is streaming live images.


In [5]:
# Define speed constants
PREFERRED_SPEED = 20  # Target speed in some units (e.g., km/h or m/s)
SPEED_THRESHOLD = 2  # Speed tolerance before adjustments

# Adding parameters to display text on the image
font = cv2.FONT_HERSHEY_SIMPLEX  # Font type for OpenCV
# Defining positions for telemetry display on the screen
org_speed = (30, 30)  # Current speed display position
org_steering = (30, 50)  # Future steering angle display position
org_telemetry = (30, 70)  # Other telemetry outputs

# Text properties
fontScale = 0.8
color = (255, 255, 255)  # White color for text
thickness = 2  # Line thickness for text

# Function to maintain speed
def maintain_speed(current_speed):
    """
    Adjust throttle to maintain desired speed.

    Args:
        current_speed (float): The current speed of the vehicle.

    Returns:
        float: Throttle value to adjust speed.
    """
    if current_speed < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.8  # Full gas (adjust this value as needed)
    elif current_speed > PREFERRED_SPEED + SPEED_THRESHOLD:
        return 0.4  # Gentle throttle to slow down
    else:
        return 0.6  # Maintain steady speed

# Example usage of displaying telemetry on an image
def display_telemetry(image, current_speed, steering_angle):
    """
    Displays telemetry data on an image.

    Args:
        image (numpy array): The image where telemetry is displayed.
        current_speed (float): The current speed of the vehicle.
        steering_angle (float): The current steering angle of the vehicle.

    Returns:
        numpy array: The image with telemetry overlaid.
    """
    speed_text = f"Speed: {current_speed:.2f}"
    steering_text = f"Steering: {steering_angle:.2f}"
    telemetry_text = "Telemetry: OK"  # Replace with other telemetry info if needed

    # Put text on the image
    cv2.putText(image, speed_text, org_speed, font, fontScale, color, thickness)
    cv2.putText(image, steering_text, org_steering, font, fontScale, color, thickness)
    cv2.putText(image, telemetry_text, org_telemetry, font, fontScale, color, thickness)

    return image


In [6]:
# Mini driving straight demo

import math
import cv2

# Assuming previous functions and setup are already done
cv2.namedWindow("RGB Camera", cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera', camera_data['image'])

# Main loop
quit = False

while not quit:
    # CARLA Tick
    world.tick()

    # Check for quit command
    if cv2.waitKey(1) == ord('q'):
        quit = True
        break

   # Get the current image from the camera
    image = camera_data["image"]

    # Make the image writable (convert to a writable NumPy array)
    image = image.copy()

    steering_angle = 0 # we do not have it yet

    # to get speed we need to use "get velocity" function
    v = vehicle.get_velocity()  

    # if velocity is a vector in 3d, then speed is the hypothenus in a right triangle
    # so, 3.6 is a conversion factor from meters per second to kmph
    # e.g. kmph is 1000 meters and one hour is 60 min with 60 sec = 3600 sec
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2), 0)

    # Display speed on the camera feed
    image = cv2.putText(image, 'Speed: '+str(int(speed))+ 'kmph', org_speed, font, 
                        fontScale, color, thickness, cv2.LINE_AA)

    # Calculate throttle using maintain_speed function
    estimated_throttle = maintain_speed(speed)

    # Apply control to the vehicle
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, 
                                               steer=steering_angle))  # steer=0.0 for straight driving

    # Show the camera feed with speed overlay
    cv2.imshow("RGB Camera", image)

# Clean up
cv2.destroyAllWindows()
camera.stop()

# Destroy all vehicles
for actor in world.get_actors().filter('vehicle.*'):
    actor.destroy()

# Destroy all sensors
for sensor in world.get_actors().filter('sensor.*'):
    sensor.destroy()
